Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (18724, 28, 28, 1) (18724, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [26]:
batch_size = 60
patch_size = 5
depth = 32
num_hidden = 1024

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth*2], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth*2]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size / 4 * image_size / 4 * depth*2, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  # Dropout.
  keep_prob = tf.placeholder("float")
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    hidden = tf.nn.max_pool(hidden, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1],padding = 'SAME')
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    hidden = tf.nn.max_pool(hidden, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1],padding = 'SAME')
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    hidden = tf.nn.dropout(hidden, keep_prob)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))
  saver = tf.train.Saver()

In [27]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  saver.restore(session,'conv_param3-10000')
  print "Initialized"
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 200 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict={keep_prob : 1}), valid_labels)
    if (step % 5000 == 0):
        saver.save(session,'conv_param4',global_step = step)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob : 1}), test_labels)

Initialized
Minibatch loss at step 0 : 0.255949
Minibatch accuracy: 91.7%
Validation accuracy: 89.7%
Minibatch loss at step 200 : 0.285924
Minibatch accuracy: 86.7%
Validation accuracy: 89.7%
Minibatch loss at step 400 : 0.420775
Minibatch accuracy: 88.3%
Validation accuracy: 89.7%
Minibatch loss at step 600 : 0.225016
Minibatch accuracy: 91.7%
Validation accuracy: 89.8%
Minibatch loss at step 800 : 0.373808
Minibatch accuracy: 88.3%
Validation accuracy: 89.8%
Minibatch loss at step 1000 : 0.362998
Minibatch accuracy: 85.0%
Validation accuracy: 89.9%
Minibatch loss at step 1200 : 0.380673
Minibatch accuracy: 90.0%
Validation accuracy: 89.9%
Minibatch loss at step 1400 : 0.26874
Minibatch accuracy: 91.7%
Validation accuracy: 89.8%
Minibatch loss at step 1600 : 0.217957
Minibatch accuracy: 96.7%
Validation accuracy: 89.8%
Minibatch loss at step 1800 : 0.206552
Minibatch accuracy: 93.3%
Validation accuracy: 90.0%
Minibatch loss at step 2000 : 0.319201
Minibatch accuracy: 91.7%
Validation 

In [15]:
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  saver.restore(session,'conv_param-2000')
  print "Initialized"
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob : 1}), test_labels)

Initialized
Test accuracy: 91.7%


In [16]:
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  saver.restore(session,'conv_param-4000')
  print "Initialized"
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob : 1}), test_labels)

Initialized
Test accuracy: 92.9%


In [17]:
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  saver.restore(session,'conv_param-6000')
  print "Initialized"
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob : 1}), test_labels)

Initialized
Test accuracy: 93.5%


In [18]:
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  saver.restore(session,'conv_param-8000')
  print "Initialized"
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob : 1}), test_labels)

Initialized
Test accuracy: 94.1%


In [19]:
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  saver.restore(session,'conv_param-10000')
  print "Initialized"
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob : 1}), test_labels)

Initialized
Test accuracy: 94.3%


In [22]:
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  saver.restore(session,'conv_param2-10000')
  print "Initialized"
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob : 1}), test_labels)

Initialized
Test accuracy: 94.7%


In [25]:
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  saver.restore(session,'conv_param3-10000')
  print "Initialized"
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob : 1}), test_labels)

Initialized
Test accuracy: 95.2%


In [28]:
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  saver.restore(session,'conv_param4-10000')
  print "Initialized"
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob : 1}), test_labels)

Initialized
Test accuracy: 95.5%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---